In [25]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F


In [30]:
#2.1
#convert pil image to tensor and centralize data. The data range is from -0.5 to 0.5
mean = 0.5
sd = 0.5
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((mean,), (sd,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=4)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=4)

In [37]:
#2.2
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #Convolution layer with 32 kernels of size 3x3
        self.conv1 = nn.Conv2d(1, 32, (3,3))
        #Convolution layer with 64 kernels of size 3x3
        self.conv2 = nn.Conv2d(32, 64, (3,3))
        #maxpool layer with kernel 2x2
        self.pool = nn.MaxPool2d(2,2)
        #Convulaton layer with 64 kernels of size 3x3
        self.conv3 = nn.Conv2d(64, 64, (3,3))
        #A convolution layer with 64 kernels of size 3×3.
        self.conv4 = nn.Conv2d(64, 64, (3,3))
        #A flattening layer. (This layer resizes 2D feature map to a feature vector. The length of this feature vector should be 4096.)
        self.flat = nn.Flatten()
        #A Linear layer with output size of 10.
        self.fc1 = nn.Linear(4096, 10)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.flat(x)
        x = self.fc1(x)
        return x


net = Net()

In [38]:
#2.3
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

In [40]:
#2.4, 2.5
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 1000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')

# testing of model
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

KeyboardInterrupt: ignored